In [12]:
import requests
from azure.storage.blob import BlobClient
import json

API_KEY = "XYZ"
Connection_str= "ABC"
container = "solardata"

with open("india_locations.json") as f:
    india_data = json.load(f)
states = india_data["locations"]

for state in states:
    state_name = state["state"]
    for city_obj in state["cities"]:
        city = city_obj["city"]
        lat = city_obj["lat"]
        lon = city_obj["lon"]

        url = (
            "https://developer.nrel.gov/api/pvwatts/v8.json"
            f"?api_key={API_KEY}"
            f"&lat={lat}&lon={lon}"
            "&system_capacity=1.0"
            "&module_type=0"
            "&array_type=1"
            f"&tilt={lat}"
            "&azimuth=180"
            "&losses=14"
        )

        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            data["metadata"] = {
                "state": state_name,
                "city": city,
                "lat": lat,
                "lon": lon
            }

            city_clean = (
                city.replace(" ", "_")
                    .replace("(", "")
                    .replace(")", "")
                    .replace("/", "_")
            )
            filename = f"{city_clean}.json"

            state_clean = state_name.replace(" ", "_")
            blob_path = f"raw/{state_clean}/{filename}"

            blob = BlobClient.from_connection_string(
                conn_str=Connection_str,
                container_name=container,
                blob_name=blob_path
            )
            blob.upload_blob(json.dumps(data).encode("utf-8"), overwrite=True)

        else:
            print(f"Failed for {city}, {state_name}: {response.status_code}")
